In [ ]:
!pip install scipy==1.10.1 scikit-image==0.19.3 vit_keras==0.1.2

In [ ]:
import sys
import io
import gc
import ctypes
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import cv2
import os
import cv2
import random
import shutil
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.client import device_lib
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from PIL import Image
from sklearn.preprocessing import LabelEncoder
import numpy as np
from vit_keras import vit, utils
import lime
import skimage
import shap
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from lime.lime_image import LimeImageExplainer
import ctypes



def preprocess_image(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = tf.image.resize(image, [resize_size, resize_size], method=tf.image.ResizeMethod.BILINEAR) #크기 조절
    image = tf.image.central_crop(image, central_fraction=crop_size / resize_size) #중앙 224x224
    image = tf.math.divide(image, 255.0) #normalize
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    image = (image - mean) / std #다 normalize

    return image


def resize_and_crop(image, resize_size=256, crop_size=224):
    # Resize with bilinear interpolation
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    resized_image = tf.image.resize(image, [resize_size, resize_size], method=tf.image.ResizeMethod.BILINEAR)
    cropped_image = tf.image.central_crop(resized_image, central_fraction=crop_size / resize_size)
    image = tf.math.divide(cropped_image, 255.0) #normalize

    return image

def ifgsm_attack(model, image, label, epsilon=0.001, num_iter=30, clip_min=0.0, clip_max=1.0):
    adv_image = tf.identity(image)

    for _ in range(num_iter):
        with tf.GradientTape() as tape:
            tape.watch(adv_image)
            prediction = model(preprocess_for_attack(adv_image))
            loss = tf.keras.losses.sparse_categorical_crossentropy(label, prediction)

        gradient = tape.gradient(loss, adv_image)
        perturbation = epsilon * tf.sign(gradient)

        adv_image = tf.clip_by_value(adv_image + perturbation, clip_min, clip_max)

    #adv_image_np = adv_image.numpy().squeeze()  # Squeeze to remove channel dimension
    del gradient, perturbation

    return adv_image

import tensorflow as tf
import numpy as np

def ifgsm_attack_transparency(model, masked_image, original_image, label, epsilon=0.001, num_iterations=30):
    # Ensure the input is a float type
    masked_image = np.expand_dims(masked_image, axis=0)
    perturbed_image = tf.cast(masked_image, tf.float32)

    # Loop over the number of iterations
    for i in range(num_iterations):
        with tf.GradientTape() as tape:
            tape.watch(perturbed_image)
            prediction = model(perturbed_image[..., :3])  # Exclude alpha channel for model prediction
            loss = tf.keras.losses.sparse_categorical_crossentropy(label, prediction)
            
        # Calculate gradients
        gradient = tape.gradient(loss, perturbed_image)[..., :3]  # Exclude alpha channel in gradient

        # Apply gradients only where the alpha channel is non-zero
        alpha_channel = perturbed_image[..., -1:]
        masked_gradient = gradient * tf.cast(alpha_channel > 0, tf.float32)

        # Apply the perturbation
        perturbed_image = perturbed_image[..., :3] + epsilon * tf.sign(masked_gradient)
        perturbed_image = tf.concat([perturbed_image, alpha_channel], axis=-1)  # Re-add the alpha channel

        # Ensure the perturbed image is still valid
        perturbed_image = tf.clip_by_value(perturbed_image, 0, 255)
        
    # Replace the areas not attacked with original image
    # Replace the areas not attacked with the original image
    mask = alpha_channel > 0
    mask_float = tf.cast(mask, tf.float32)  # Convert the boolean mask to float

    # Ensure original_image and perturbed_image are of the same type (e.g., float32)
    original_image_float = tf.cast(original_image, tf.float32)
    perturbed_image_float = tf.cast(perturbed_image[..., :3], tf.float32)

    # Combine the images using the mask
    result_image = original_image_float * (1 - mask_float) + perturbed_image_float * mask_float
    
    del perturbed_image, mask,original_image_float, perturbed_image_float

    return result_image


def preprocess_for_attack(image):

    # 이미지 정규화
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    image = (image - mean) / std

    return image


def plot_images(original, temp, mask, model_name, model_type, file_name, label, model):
    width_in_inches = height_in_inches = 224 / 100
    save_paths = [f'{model_name}/{model_type}/original/', f'{model_name}/{model_type}/superpixel/', f'{model_name}/{model_type}/pos_neg/'
                 , f'{model_name}/{model_type}/ifgsm_super_pixel/', f'{model_name}/{model_type}/ifgsm_full/',
                 f'{model_name}/{model_type}/ifgsm_negative/', f'{model_name}/{model_type}/ifgsm_pos+neg/',
                  f'{model_name}/{model_type}/neg/', f'{model_name}/{model_type}/combine/']
    
    for save_path in save_paths:
        if not os.path.exists(save_path):
            os.makedirs(save_path)

    ##original image###
    plt.figure(figsize=(width_in_inches, height_in_inches))
    plt.imshow(original)
    plt.axis('off')
    plt.savefig(os.path.join(f'{model_name}/{model_type}/original/', f'{file_name}'), bbox_inches='tight', pad_inches=0)
    plt.close()
    
    
    ###super pixel + attack super pixel ###
    masked_positive = np.copy(original)
    masked_positive = np.concatenate((masked_positive, np.ones((*masked_positive.shape[:-1], 1), dtype=masked_positive.dtype) * 255), axis=-1)  # Add alpha channel
    masked_positive[mask <= 0, -1] = 0
    
    plt.imshow(masked_positive)
    plt.axis('off')
    plt.savefig(os.path.join(f'{model_name}/{model_type}/superpixel/', f'{file_name}'), bbox_inches='tight', pad_inches=0)
    plt.close()
    
    masked_positive_adv = ifgsm_attack_transparency(model, masked_positive, original, label)
    masked_positive_adv = np.squeeze(masked_positive_adv, axis=0)

    plt.imshow(masked_positive_adv)
    plt.axis('off')
    plt.savefig(os.path.join(f'{model_name}/{model_type}/ifgsm_super_pixel/', f'{file_name}'), bbox_inches='tight', pad_inches=0)
    plt.close() 
    
    
    ##attack negative pixel###
    masked_negative = np.copy(original)
    masked_negative = np.concatenate((masked_negative, np.ones((*masked_negative.shape[:-1], 1), dtype=masked_negative.dtype) * 255), axis=-1)  # Add alpha channel
    masked_negative[mask >= 0, -1] = 0
    
    plt.imshow(masked_negative)
    plt.axis('off')
    plt.savefig(os.path.join(f'{model_name}/{model_type}/neg/', f'{file_name}'), bbox_inches='tight', pad_inches=0)
    plt.close()

    masked_negative_adv = ifgsm_attack_transparency(model, masked_negative, original, label)
    masked_negative_adv = np.squeeze(masked_negative_adv, axis=0)

    plt.imshow(masked_negative_adv)
    plt.axis('off')
    plt.savefig(os.path.join(f'{model_name}/{model_type}/ifgsm_negative/', f'{file_name}'), bbox_inches='tight', pad_inches=0)
    plt.close()

    
    #attack pos+negative pixel###
    masked_combined = np.copy(original)
    masked_combined = np.concatenate((masked_combined, np.ones((*masked_combined.shape[:-1], 1), dtype=masked_negative.dtype) * 255), axis=-1)  # Add alpha channel
    masked_combined[mask == 0, -1] = 0
    
    plt.imshow(masked_combined)
    plt.axis('off')
    plt.savefig(os.path.join(f'{model_name}/{model_type}/combine/', f'{file_name}'), bbox_inches='tight', pad_inches=0)
    plt.close()

    masked_combined_adv = ifgsm_attack_transparency(model, masked_combined, original, label)
    masked_combined_adv = np.squeeze(masked_combined_adv, axis=0)

    plt.imshow(masked_combined_adv)
    plt.axis('off')
    plt.savefig(os.path.join(f'{model_name}/{model_type}/ifgsm_pos+neg/', f'{file_name}'), bbox_inches='tight', pad_inches=0)
    plt.close()

    # Lime Mask를 원본 이미지에 적용하여 긍부정 시각화
    masked_negative = np.zeros_like(original)
    masked_negative[mask < 0] = [255, 0, 0]
    

    # Create a new image for positive parts (green color)
    masked_positive = np.zeros_like(original)
    masked_positive[mask > 0] = [0, 255, 0]

    # Combine the positive and negative images
    combined_image = original + masked_negative + masked_positive

    # Display the result with larger size
    plt.imshow(combined_image)
    plt.axis('off')
    plt.savefig(os.path.join(f'{model_name}/{model_type}/pos_neg/', f'{file_name}'), bbox_inches='tight', pad_inches=0)
    plt.close()
    
    original_image = np.expand_dims(original, axis=0)
    original_attack = ifgsm_attack(model, original_image, label)
    original_attack = np.squeeze(original_attack, axis=0)
    plt.imshow(original_attack)
    plt.axis('off')
    plt.savefig(os.path.join(f'{model_name}/{model_type}/ifgsm_full/', f'{file_name}'), bbox_inches='tight', pad_inches=0)
    plt.close()
    
    del masked_positive, masked_positive_adv, masked_negative, masked_negative_adv, masked_combined, masked_combined_adv
    del combined_image, original_image, original_attack
    
    plt.cla()
    plt.clf()
    
def iFGSM_and_LIME(df, i, model):
    img = cv2.imread(df['path'].iloc[i])
    selected_image = preprocess_image(img)
    y = 0 if df['label'].iloc[i]=='dogs' else 1
    Original_image = resize_and_crop(img)
    selected_image = np.expand_dims(selected_image, axis=0)
    prediction = model.predict(selected_image)
    predicted_class = np.argmax(prediction)
    explainer = lime.lime_image.LimeImageExplainer(feature_selection='auto')
    explanation = explainer.explain_instance(selected_image[0], model.predict, top_labels=1, hide_color=0, num_samples=100)
    temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=10, hide_rest=False)
    plot_images(Original_image, temp, mask, model_name, df['divide'].iloc[i], "/".join(df['path'].iloc[i].split('/')[4:]).replace("/","-"), predicted_class, model)
    del Original_image, selected_image, explainer, explanation, temp, mask, img
    return ['dogs', 'cats'][predicted_class], df['label'].iloc[i]
    
def file_split(range_, df, model):
    for i in tqdm(range_):
        full_prediction, full_true = iFGSM_and_LIME(df, i, model)
        print(full_prediction, full_true)

In [ ]:
if __name__ == '__main__':
    original_stdout = sys.stdout
    sys.stdout = io.StringIO()
    ctypes.CDLL("libc.so.6").malloc_trim(0) 
    count = 0
    start = 800
    resize_size = 256
    crop_size = 224

    while True:
        end = 1000
        df = pd.read_csv('/kaggle/input/dog-cat-pandas/cat_dog_df.csv', index_col=0)
        df = df[df['divide']=='test']
        model_name = 'vit'
        model = keras.models.load_model('/kaggle/input/dog-and-cat-classifier/'+model_name+"_best_model.h5", compile=False)
        model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])
        file_split(range(start, end), df, model)
        gc.collect()
        ctypes.CDLL("libc.so.6").malloc_trim(0)
        start += 500
        break

In [ ]:
import os
import shutil
from IPython.display import FileLink

#sys.stdout = original_stdout
os.chdir('/kaggle/working')
archive_name = 'kaggle_working.tar.gz'
shutil.make_archive(archive_name.replace('.tar.gz', ''), 'gztar', root_dir='.', base_dir='.')
FileLink(archive_name)

In [ ]:
def preprocess_image(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = tf.image.resize(image, [224, 224], method=tf.image.ResizeMethod.BILINEAR) #크기 조절
    image = tf.math.divide(image, 255.0) #normalize
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    image = (image - mean) / std #다 normalize

    return image

In [ ]:
def check_accuracy(test_dir):
    for model_name in ['vgg']:
        model = keras.models.load_model('/kaggle/input/dog-and-cat-classifier/'+model_name+"_best_model.h5", compile=False)
        model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])
        for data_dir in [test_dir]:
            folderPath = data_dir
            for i in range(988, 1038):
                j = "-".join(df['path'].iloc[i].split("/")[4:])
                print(os.path.join(folderPath, j))
                img = cv2.imread(os.path.join(folderPath, j))  # Read image
                img = preprocess_image(img)
                img_batch = np.expand_dims(img, axis=0)  # Expand dimensions
                y_pred = model.predict(img_batch)  # Use img_batch here
                y_pred_single_label = np.argmax(y_pred, axis=1)
                print(y_pred_single_label)
                tf_list.append(classes[int(y_pred_single_label)]==j.replace("-", ".").split(".")[-4])

    counts = Counter(tf_list)
    for element, count in counts.items():
        print(element, ":", count)

In [ ]:
from collections import Counter

classes = ['dogs', 'cats']
tf_list = []
test_dir = '//kaggle/input/for-test/vgg/train/ifgsm_negative'

check_accuracy(test_dir)